In [ ]:
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from definitions import ROOT_DIR


In [ ]:
bids_df_path = ROOT_DIR / "data" / "online_rl_data" / "bids.parquet"
pvalues_df_path = ROOT_DIR / "data" / "online_rl_data" / "pvalues.parquet"

bids_df = pd.read_parquet(bids_df_path)
bids_df["bid"] = bids_df["bid"].apply(np.stack)
bids_df["isExposed"] = bids_df["isExposed"].apply(np.stack)
bids_df["cost"] = bids_df["cost"].apply(np.stack)
pvalues_df = pd.read_parquet(pvalues_df_path)

In [ ]:
bids_df.head()

In [ ]:
pvalues_df.head()

In [ ]:
# advertiser_idx = 4
# target_cpa = 9

# ad_pvalues_df = pvalues_df[pvalues_df["advertiserNumber"] == advertiser_idx]

# min_cost_list = []
# pvalues_list = []
# for ts in ad_pvalues_df.timeStepIndex.unique():
#     ts_min_cost = bids_df[bids_df.timeStepIndex == ts].cost.item()[:, 0]
#     min_cost_list.append(ts_min_cost)
#     ts_pvalues = ad_pvalues_df[ad_pvalues_df.timeStepIndex == ts].pValue.item()
#     pvalues_list.append(ts_pvalues)
# pvalues = np.concatenate(pvalues_list)
# min_cost = np.concatenate(min_cost_list)

# # Plot the cost and the pvalues in the lists and color them depending on the ts
# for ts, (ts_pvalues, ts_min_cost) in enumerate(zip(pvalues_list, min_cost_list)):
#     fig, ax = plt.subplots()
#     plt.plot(pvalues, target_cpa * pvalues, "r-")
#     ax.scatter(ts_pvalues, ts_min_cost, alpha=0.1, s=1, label=f"ts={ts}")
#     ax.set_xlim(0, pvalues.max())
#     ax.set_ylim(0, min_cost.max())
#     ax.set_xlabel("pValues")
#     ax.set_ylabel("Least winning cost")
#     ax.set_title(f"ts {ts}")

In [ ]:
target_cpa = 9
advertiser_idx = 0

# for advertiser_idx in range(28):
for period in pvalues_df.deliveryPeriodIndex.unique():
    ad_pvalues_df = pvalues_df[(pvalues_df["advertiserNumber"] == advertiser_idx) & (pvalues_df["deliveryPeriodIndex"] == period)]

    min_cost_list = []
    pvalues_list = []
    for ts in ad_pvalues_df.timeStepIndex.unique():
        ts_min_cost = bids_df[(bids_df.timeStepIndex == ts) & (bids_df.deliveryPeriodIndex == period)].cost.item()[:, 0]
        min_cost_list.append(ts_min_cost)
        ts_pvalues = ad_pvalues_df[ad_pvalues_df.timeStepIndex == ts].pValue.item()
        pvalues_list.append(ts_pvalues)
    pvalues = np.concatenate(pvalues_list)
    min_cost = np.concatenate(min_cost_list)

    pv_over_cost  = []
    for ts_pvalues, ts_min_cost in zip(pvalues_list, min_cost_list):
        pv_over_cost.append(ts_pvalues / ts_min_cost)
        
    fig, ax = plt.subplots()
    # ax.plot([np.mean(v) for v in pv_over_cost])
    # plot mean and std of pv_over_cost
    # ax.errorbar(np.arange(48), [np.mean(v) for v in pv_over_cost], yerr=[np.std(v) for v in pv_over_cost])
    
    # plot boxplot of pv_over_cost
    ax.boxplot(pv_over_cost)
    ax.plot(np.arange(48), np.ones(48) / target_cpa, "r-")
    ax.set_title(f"Advertiser {advertiser_idx} period {period}")
    fig.show()
